# Model Performance

This notebook will allow us to compare trained models on the test set. The analysis and metrics are standard for classification models, and the code is based on week 2 of Coursera's [AI for Medical Diagnosis](https://www.coursera.org/learn/ai-for-medical-diagnosis) 

TODO
- Move the functions out of this notebook and put them in the utils folder
- Add confusion matrix

## Data Processing

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

from utils.metrics_utils import get_performance_metrics, print_confidence_intervals, get_curve

Get a dictionary of images for our dataset and create a lookup table for readable names for our classes

In [ ]:
base_dir = os.path.join('..', 'data')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

image_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_dir, '*', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
print(f'There are {len(image_path_dict)} images in our dataset')

Here we will read and process the data. This will help later with creating labels.

In [ ]:
skin_df = pd.read_csv(os.path.join(base_dir, 'datasets_54339_104884_HAM10000_metadata.csv'))

# Creating New Columns for better readability

skin_df['path'] = skin_df['image_id'].map(image_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [ ]:
skin_df.head()

## Data Quality

Look for duplicate images from patients and make sure datasets are stratified

In [ ]:
df = skin_df.groupby('lesion_id').count()

In [ ]:
print(f'Original dataset had {skin_df.shape[0]} records, there are {df.shape[0]} unique lesions')

## Create Train, Test, and Val Sets

- TODO Use TF Records and tf dataset to store training dataset as an alternative to data generator below

We see that there are numerous images taken for some patients, therefore we will choose a single image from each patient. Then we will take a stratified sample across our target variable in order to create our train test and validation directories.

First create a dataframe containing a single image from each patient. Note that we could also try including these duplicates, just making sure that when we split our dataset we keep patients in a single train, test, or val set.

In [ ]:
# Set a seed (random_state) for reproducibility and deterministic train/val/test sets
df_dataset = skin_df.sample(frac=1, random_state=123).drop_duplicates(subset='lesion_id').copy()
df_dataset.reset_index(drop=True, inplace=True)

In [ ]:
CLASS_LABELS = [
    'nv' ,
    'mel', 
    'bkl', 
    'bcc',
    'akiec',
    'vasc',
    'df',
]

In [ ]:
df_dataset.head()

In [ ]:
img_shape = np.asarray(Image.open(df_dataset['path'][0])).shape
print('Image shape:', img_shape)

Create stratified train/test/val sets

In [ ]:
X = df_dataset['path']
y = df_dataset['cell_type_idx']

Set a seed (random_state) for reproducibility and deterministic train/val/test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.111, random_state=321)

In [ ]:
train_data = pd.DataFrame({
    'path': X_train,
    'cell_type_idx': y_train
})

In [ ]:
def convert_image_to_array(path):
    return np.asarray(Image.open(path), dtype=np.float32)

In [ ]:
def create_model_file(X_path, y):
    """
    X_path: (pandas series) contains the file paths to the images
    y: (pandas series of type int) the target label
    
    return a pair of numpy arrays representing (features, target)
    """
    
    X = X_path.apply(convert_image_to_array)
    X /= 255.
    X = X.values
    X = list(X)
    X = np.array(X)
    
    y = y.map(lambda y: to_categorical(y, num_classes=len(CLASS_LABELS)))
    y = y.values
    y = list(y)
    y = np.array(y)
    
    return (X, y)

In [ ]:
def model_predict(path, model):
    x = convert_image_to_array(path=path)
    x /= 255.
    x = np.expand_dims(x, axis=0)
    return model.predict(x)

In [ ]:
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
#test_data = create_model_file(X_path=X_test, y=y_test)

## Import the model

Here we import a fitted model and we will select the layers on which the localization will be based.

In [ ]:
MODEL_NAME = 'model_DN201_w_ClssWgt_06-0.5685'

In [ ]:
model = tf.keras.models.load_model(filepath=f'../serialized_models/{MODEL_NAME}.h5')

In [ ]:
SAMPLE_NUM = 0
SAMPLE_PATH = X_test[SAMPLE_NUM]
img = np.asarray(Image.open(SAMPLE_PATH))

label_int = y_test[SAMPLE_NUM]
label_abbreviation = CLASS_LABELS[label_int]
print(f'The image contains {lesion_type_dict[label_abbreviation]}')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.imshow(img)
ax.axis('off')
ax.set_aspect('auto')

plt.show() 

In [ ]:
model_predict(path=SAMPLE_PATH, model=model)

In [ ]:
PRED_LABELS = [l + "_pred" for l in CLASS_LABELS]

Get all model predictions

In [ ]:
all_model_preds = pd.DataFrame(0, index=np.arange(len(X_test)), columns=PRED_LABELS)

In [ ]:
for i, path in enumerate(X_test):
    all_model_preds.iloc[i, :] = model_predict(path=path, model=model)[0]

In [ ]:
results = pd.get_dummies(data=y_test)

In [ ]:
col_rename = {i:label for i, label in enumerate(CLASS_LABELS)}

In [ ]:
results.rename(columns=col_rename, inplace=True)

In [ ]:
results = pd.concat([results, all_model_preds], axis=1)

In [ ]:
results

In [ ]:
y = results[CLASS_LABELS].values
pred = results[PRED_LABELS].values

In [ ]:
plt.xticks(rotation=90)
plt.bar(x = CLASS_LABELS, height= y.sum(axis=0));

All the evaluatin metrics we will use can be computed from the number of true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN). Below these are defined along with a small unit test.

In [ ]:
def true_positives(y, pred, th=0.5):
    """
    Count true positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TP (int): true positives
    """
    TP = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th

    # compute TP
    TP = np.sum((y == 1) & (thresholded_preds == 1))
    
    return TP

def true_negatives(y, pred, th=0.5):
    """
    Count true negatives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TN (int): true negatives
    """
    TN = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th
    
    # compute TN
    TN = np.sum((y == 0) & (thresholded_preds == 0))
    
    return TN

def false_positives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FP (int): false positives
    """
    FP = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th

    # compute FP
    FP = np.sum((y == 0) & (thresholded_preds == 1))
    
    return FP

def false_negatives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FN (int): false negatives
    """
    FN = 0
    
    # get thresholded predictions
    thresholded_preds = pred >= th
    
    # compute FN
    FN = np.sum((y == 1) & (thresholded_preds == 0))
    
    return FN

In [ ]:
df = pd.DataFrame({'y_test': [1,1,0,0,0,0,0,0,0,1,1,1,1,1],
                   'preds_test': [0.8,0.7,0.4,0.3,0.2,0.5,0.6,0.7,0.8,0.1,0.2,0.3,0.4,0],
                   'category': ['TP','TP','TN','TN','TN','FP','FP','FP','FP','FN','FN','FN','FN','FN']
                  })
print(df)

In [ ]:
#y_test = np.array([1, 0, 0, 1, 1])
y_test = df['y_test']

#preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
preds_test = df['preds_test']

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"""Our functions calcualted: 
TP: {true_positives(y_test, preds_test, threshold)}
TN: {true_negatives(y_test, preds_test, threshold)}
FP: {false_positives(y_test, preds_test, threshold)}
FN: {false_negatives(y_test, preds_test, threshold)}
""")

print("Expected results")
print(f"There are {sum(df['category'] == 'TP')} TP")
print(f"There are {sum(df['category'] == 'TN')} TN")
print(f"There are {sum(df['category'] == 'FP')} FP")
print(f"There are {sum(df['category'] == 'FN')} FN")

In [ ]:
get_performance_metrics(y, pred, CLASS_LABELS)

In [ ]:
def get_accuracy(y, pred, th=0.5):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        accuracy (float): accuracy of predictions at threshold
    """
    accuracy = 0.0
    
    # get TP, FP, TN, FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # Compute accuracy using TP, FP, TN, FN
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    return accuracy

In [ ]:
# Test
print("Test case:")

y_test = np.array([1, 0, 0, 1, 1])
print('test labels: {y_test}')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}')

threshold = 0.5
print(f"threshold: {threshold}")

print(f"computed accuracy: {get_accuracy(y_test, preds_test, threshold)}")

#### Expected output:

```Python
test labels: {y_test}
test predictions: [0.8 0.8 0.4 0.6 0.3]
threshold: 0.5
computed accuracy: 0.6
```

In [ ]:
get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy)

In [ ]:
def get_prevalence(y):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
    Returns:
        prevalence (float): prevalence of positive cases
    """
    prevalence = 0.0
    
    prevalence = np.sum(y) / len(y)
    
    return prevalence

In [ ]:
# Test
print("Test case:\n")

y_test = np.array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1])
print(f'test labels: {y_test}')

print(f"computed prevalence: {get_prevalence(y_test)}")

In [ ]:
get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy, prevalence=get_prevalence)

In [ ]:
def get_sensitivity(y, pred, th=0.5):
    """
    Compute sensitivity of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        sensitivity (float): probability that our test outputs positive given that the case is actually positive
    """
    sensitivity = 0.0
    
    # get TP and FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TP and FN to compute sensitivity
    sensitivity = TP / (TP + FN)
    
    return sensitivity

def get_specificity(y, pred, th=0.5):
    """
    Compute specificity of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        specificity (float): probability that the test outputs negative given that the case is actually negative
    """
    specificity = 0.0
    
    # get TN and FP using our previously defined functions
    TN = true_negatives(y, pred, th)
    FP = false_positives(y, pred, th)
    
    # use TN and FP to compute specificity 
    specificity = TN / (TN + FP)
    
    return specificity

In [ ]:
# Test
print("Test case")

y_test = np.array([1, 0, 0, 1, 1])
print(f'test labels: {y_test}\n')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}\n')

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"computed sensitivity: {get_sensitivity(y_test, preds_test, threshold):.2f}")
print(f"computed specificity: {get_specificity(y_test, preds_test, threshold):.2f}")

#### Expected output:

```Python
Test case
test labels: [1 0 0 1 1]

test predictions: [0.8 0.8 0.4 0.6 0.3]

threshold: 0.5

computed sensitivity: 0.67
computed specificity: 0.50

```

In [ ]:
get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy, prevalence=get_prevalence, sens=get_sensitivity, spec=get_specificity)

In [ ]:
def get_ppv(y, pred, th=0.5):
    """
    Compute PPV of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        PPV (float): positive predictive value of predictions at threshold
    """
    PPV = 0.0
    
    # get TP and FP using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)

    # use TP and FP to compute PPV
    PPV = TP / (TP + FP)
    
    return PPV

def get_npv(y, pred, th=0.5):
    """
    Compute NPV of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        NPV (float): negative predictive value of predictions at threshold
    """
    NPV = 0.0
    
    # get TN and FN using our previously defined functions
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TN and FN to compute NPV
    NPV = TN / (TN + FN)
    
    return NPV

In [ ]:
# Test
print("Test case:\n")

y_test = np.array([1, 0, 0, 1, 1])
print(f'test labels: {y_test}')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}\n')

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"computed ppv: {get_ppv(y_test, preds_test, threshold):.2f}")
print(f"computed npv: {get_npv(y_test, preds_test, threshold):.2f}")

#### Expected output:

```Python
Test case:

test labels: [1 0 0 1 1]
test predictions: [0.8 0.8 0.4 0.6 0.3]

threshold: 0.5

computed ppv: 0.67
computed npv: 0.50
```

In [ ]:
get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy, prevalence=get_prevalence, sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv)

In [ ]:
get_curve(y, pred, CLASS_LABELS)

In [ ]:
from sklearn.metrics import roc_auc_score
get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score)

In [ ]:
def bootstrap_auc(y, pred, classes, bootstraps = 100, fold_size = 1000):
    statistics = np.zeros((len(classes), bootstraps))

    for c in range(len(classes)):
        df = pd.DataFrame(columns=['y', 'pred'])
        df.loc[:, 'y'] = y[:, c]
        df.loc[:, 'pred'] = pred[:, c]
        # get positive examples for stratified sampling
        df_pos = df[df.y == 1]
        df_neg = df[df.y == 0]
        prevalence = len(df_pos) / len(df)
        for i in range(bootstraps):
            # stratified sampling of positive and negative examples
            pos_sample = df_pos.sample(n = int(fold_size * prevalence), replace=True)
            neg_sample = df_neg.sample(n = int(fold_size * (1-prevalence)), replace=True)

            y_sample = np.concatenate([pos_sample.y.values, neg_sample.y.values])
            pred_sample = np.concatenate([pos_sample.pred.values, neg_sample.pred.values])
            score = roc_auc_score(y_sample, pred_sample)
            statistics[c][i] = score
    return statistics

statistics = bootstrap_auc(y, pred, CLASS_LABELS)

In [ ]:
print_confidence_intervals(CLASS_LABELS, statistics)

In [ ]:
get_curve(y, pred, CLASS_LABELS, curve='prc')

In [ ]:
from sklearn.calibration import calibration_curve
def plot_calibration_curve(y, pred):
    plt.figure(figsize=(20, 20))
    for i in range(len(CLASS_LABELS)):
        plt.subplot(4, 4, i + 1)
        fraction_of_positives, mean_predicted_value = calibration_curve(y[:,i], pred[:,i], n_bins=20)
        plt.plot([0, 1], [0, 1], linestyle='--')
        plt.plot(mean_predicted_value, fraction_of_positives, marker='.')
        plt.xlabel("Predicted Value")
        plt.ylabel("Fraction of Positives")
        plt.title(CLASS_LABELS[i])
    plt.tight_layout()
    plt.show()

In [ ]:
plot_calibration_curve(y, pred)

In [ ]:
from sklearn.metrics import f1_score
model_performance_df = get_performance_metrics(y, pred, CLASS_LABELS, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

## Save the test set results

Save the results along with the model name so that we can compare various models

In [ ]:
model_performance_df

In [ ]:
model_performance_df.rename(index=lesion_type_dict, inplace=True)

In [ ]:
model_performance_df

In [ ]:
model_performance_df.to_csv(f'test_set_performance/test_metrics_{MODEL_NAME}.csv')